本章的目的是说明使用 scikit-learn 时出现的一些常见陷阱和反模式。它提供了不该做什么的示例，以及相应的正确示例。

scikit-learn 提供了一个数据集转换库，它可以清理（参见预处理数据）、减少（参见无监督降维）、扩展（参见内核近似）或生成（参见特征提取）特征表示。如果在训练模型时使用这些数据转换，那么它们也必须用于后续数据集，无论是测试数据还是生产系统中的数据。否则，特征空间会发生变化，模型将无法有效执行。

以下示例创建了一个具有单个特征的合成数据集：

In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

random_state = 42
X, y = make_regression(random_state=random_state, n_features=1, noise=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=random_state)

**错误的示例**

训练数据集被缩放，而不是测试数据集，因此测试数据集上的模型性能比预期的要差：

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [3]:
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
model = LinearRegression()
model.fit(X_train_transformed, y_train)
mean_squared_error(y_test, model.predict(X_test))

62.80867119249524

**正确的示例**

In [4]:
X_test_transformed = scaler.transform(X_test)
mean_squared_error(y_test, model.predict(X_test_transformed))

0.9027975466369481

**使用管道**

我们使用管道来更轻松地将转换器和估计器链接起来，减少忘记转换的可能性：

In [5]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

model.fit(X_train, y_train)
mean_squared_error(y_test, model.predict(X_test))

0.9027975466369481